In [2]:
import pandas as pd

df = pd.read_csv('reviews.csv')
df['rating'] = df['rating'].astype(dtype='int64')

In [3]:
from typing import Generator, List
import pickle

from gensim.models import Phrases
from gensim.models.phrases import Phraser
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
import spacy
from spacy.lang.en import English

In [4]:
def sentences_to_words(sentences: List[str]) -> Generator:
    for sentence in sentences:
        # https://radimrehurek.com/gensim/utils.html#gensim.utils.simple_preprocess
        yield(simple_preprocess(str(sentence), deacc=True))  # deacc=True elimina la puntuación

In [5]:
def remove_stopwords(documents: List[List[str]]) -> List[List[str]]:
    return [[word for word in simple_preprocess(str(doc)) if word not in stopwords.words('english')]
            for doc in documents]

In [6]:
def learn_bigrams(documents: List[List[str]]) -> List[List[str]]:
    # We learn bigrams
    #  https://radimrehurek.com/gensim/models/phrases.html#gensim.models.phrases.Phrases
    bigram = Phrases(documents, min_count=5, threshold=10)

    # we reduce the bigram model to its minimal functionality
    bigram_mod = Phraser(bigram)

    # we apply the bigram model to our documents
    return [bigram_mod[doc] for doc in documents]

In [7]:
def lemmatization(nlp: English, texts: List[List[str]], allowed_postags: List = None) -> List[List[str]]:
    if allowed_postags is None:
        allowed_postags = ['NOUN', 'ADJ', 'VERB', 'ADV']

    """https://spacy.io/api/annotation"""
    texts_out = []
    for sent in texts:
        doc = nlp(" ".join(sent))
        texts_out.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
    return texts_out

In [8]:
nlp = spacy.load('en_core_web_lg', disable=['parser', 'ner'])

In [9]:
def tokenize(documents: List[str]) -> List[List[str]]:

    document_words = list(sentences_to_words(documents))
    document_words = remove_stopwords(document_words)
    document_words = learn_bigrams(document_words)
    document_words = lemmatization(nlp, document_words)

    return document_words

In [10]:
tokenize_reviews = tokenize(df['review'])

In [43]:
import gensim.corpora as corpora

id2word = corpora.Dictionary(tokenize_reviews)

In [44]:
corpus = [id2word.doc2bow(text) for text in tokenize_reviews]

In [45]:
lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                           id2word=id2word,
                                           num_topics=10, 
                                           random_state=100,
                                           update_every=1,
                                           chunksize=100,
                                           passes=10,
                                           alpha='auto',
                                           per_word_topics=True)

In [46]:
from pprint import pprint

pprint(lda_model.print_topics())

[(0,
  '0.035*"treat" + 0.034*"ambiance" + 0.024*"cool" + 0.022*"step" + '
  '0.021*"rest" + 0.021*"hostel" + 0.021*"music" + 0.019*"amaze" + '
  '0.018*"afternoon" + 0.017*"touch"'),
 (1,
  '0.048*"romantic" + 0.042*"huge" + 0.038*"employee" + 0.023*"advance" + '
  '0.021*"hard" + 0.019*"impressive" + 0.017*"probably" + 0.016*"rather" + '
  '0.016*"life" + 0.015*"corridor"'),
 (2,
  '0.076*"ground" + 0.044*"take" + 0.032*"many" + 0.028*"think" + '
  '0.020*"provide" + 0.018*"door" + 0.018*"candle" + 0.017*"eat" + '
  '0.016*"leave" + 0.016*"ever"'),
 (3,
  '0.058*"gorgeous" + 0.033*"central" + 0.031*"less" + 0.028*"space" + '
  '0.027*"country" + 0.023*"business" + 0.021*"fill" + 0.020*"type" + '
  '0.020*"consider" + 0.019*"next"'),
 (4,
  '0.122*"hotel" + 0.093*"room" + 0.036*"beautiful" + 0.033*"great" + '
  '0.025*"service" + 0.021*"breakfast" + 0.020*"restaurant" + 0.016*"staff" + '
  '0.015*"comfortable" + 0.015*"location"'),
 (5,
  '0.048*"free" + 0.046*"local" + 0.036*"must" +

In [14]:
import pyLDAvis
import pyLDAvis.gensim

pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim.prepare(lda_model, corpus, id2word)
vis

/Users/rodrigo/miniconda3/envs/topic-modeling/lib/python3.8/site-packages/joblib/numpy_pickle.py:103: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))
/Users/rodrigo/miniconda3/envs/topic-modeling/lib/python3.8/site-packages/joblib/numpy_pickle.py:103: DeprecationWarning: tostring() is deprecated. Use tobytes() instead.
  pickler.file_handle.write(chunk.tostring('C'))


PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
19     0.337353  0.188746       1        1  45.533466
8      0.282888  0.182115       2        1  14.285900
18     0.220099 -0.391723       3        1  12.702916
6      0.105994 -0.093531       4        1   4.949809
17     0.008489 -0.020458       5        1   2.399790
3      0.014924 -0.024631       6        1   2.250723
4      0.020622 -0.028693       7        1   2.197289
5     -0.039809  0.004037       8        1   1.644292
1     -0.031041  0.000423       9        1   1.644149
14    -0.042860  0.005084      10        1   1.591634
9     -0.054471  0.009396      11        1   1.335494
15    -0.058248  0.010651      12        1   1.297624
13    -0.121572  0.025580      13        1   1.246784
2     -0.055138  0.009560      14        1   1.199243
12    -0.073527  0.015261      15        1   1.167818
7     -0.094327  0.020555      16        1   1.026668
16    -0.064701  0.012560      17        1   1.011841
11    -0.109429  0.023607      18        1   0.999060
0     -0.122414  0.025689      19        1   0.814468
10    -0.122833  0.025769      20        1   0.701033, topic_info=             Term          Freq         Total Category  logprob  loglift
14          hotel  29736.000000  29736.000000  Default  30.0000  30.0000
29           room  23698.000000  23698.000000  Default  29.0000  29.0000
68         ground   3163.000000   3163.000000  Default  28.0000  28.0000
34           stay  14790.000000  14790.000000  Default  27.0000  27.0000
96      excellent   2928.000000   2928.000000  Default  26.0000  26.0000
...           ...           ...           ...      ...      ...      ...
14776   columbian      0.053836      1.185601  Topic20 -11.1870   1.8683
4380   collection      0.053837      1.185637  Topic20 -11.1870   1.8683
1921      upscale      0.053837      1.185629  Topic20 -11.1870   1.8683
1389          art      0.053845      1.185914  Topic20 -11.1868   1.8682
2144       museum      0.053866      1.186631  Topic20 -11.1864   1.8680

[755 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
607      17  0.996168        able
494      15  0.998716  absolutely
663      14  0.970982          ac
685       7  0.996197      access
225      11  0.997912    actually
...     ...       ...         ...
183       1  0.013578       would
183       3  0.986217       would
1135      9  0.994794       wrong
544      12  0.997391        year
489       9  0.987691    year_old

[521 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[20, 9, 19, 7, 18, 4, 5, 6, 2, 15, 10, 16, 14, 3, 13, 8, 17, 12, 1, 11])

In [61]:
import pprint
tokens = tokenize(['this is the best hotel i have stayed'])
id2word = corpora.Dictionary(tokens)
corpus = [id2word.doc2bow(text) for text in tokens]
lda_prediction = pd.DataFrame(lda_model[corpus][0][0])
lda_prediction.columns = ['Index', 'Prob']
lda_prediction[['Prob']].idxmax() 
topics = ['Cool Music Hostel','Romantic Stay','Great to Eat','Gorgeous for Business Trip','Great Staff and Location','Top and Cozy','Old with Garden Patio','Dissapointed with Hotel','Good Place to Stay','Like a Museum']
topics[lda_prediction['Index'][7]]

'Good Place to Stay'

In [67]:
from typing import List

from gensim.models import LdaModel
from gensim import corpora
import pandas as pd


def predict(documents: List[str]):

    words = tokenize(documents)
    print(words)

    #lda_model = LdaModel.load('models/lda_model')

    topics = ['Cool Music Hostel', 'Romantic Stay', 'Great to Eat', 'Gorgeous for Business Trip',
              'Great Staff and Location', 'Top and Cozy', 'Old with Garden Patio', 'Dissapointed with Hotel',
              'Good Place to Stay', 'Like a Museum']

    predictions = []
    for document in words:
        print(document)
        id2word = corpora.Dictionary(document)
        corpus = id2word.doc2bow(document)

        lda_prediction = pd.DataFrame(lda_model[corpus][0][0])
        lda_prediction.columns = ['Index', 'Prob']
        topic = topics[lda_prediction['Index'][lda_prediction[['Prob']].idxmax()]]

        predictions.append(topic)

    return predictions


In [68]:
predict(['We were on the second floor. We arrived late at night time and the front desk had everything ready to go. The rooms were a little small but nothing to fuss over. They were clean and well kept. What a surprise when we opened the door in the morning - best view in Antigua. Breakfast was simple, and the service is wonderful. Location is just outside the main area, which means it was nice and quiet. Still very walkable to sights. I would stay here again.','We came here with a group and this was a five star surprise. Lush landscaping, wonderful beds, and close to the square  The water is drinkable but they go sell bottled water. The is a large grocery store only 3 blocks away for anything you might need '])

[['second', 'floor', 'arrive', 'late', 'night', 'time', 'front', 'desk', 'ready', 'go', 'room', 'little', 'small', 'fuss', 'clean', 'well', 'keep', 'surprise', 'open', 'door', 'morning', 'good', 'service', 'wonderful', 'location', 'main', 'area', 'mean', 'nice', 'quiet', 'still', 'walkable', 'sight', 'would', 'stay'], ['come', 'star', 'surprise', 'lush', 'landscaping', 'wonderful', 'bed', 'close', 'square', 'water', 'drinkable', 'go', 'sell', 'bottled', 'water', 'large', 'grocery', 'store', 'block', 'away', 'may', 'need']]
['second', 'floor', 'arrive', 'late', 'night', 'time', 'front', 'desk', 'ready', 'go', 'room', 'little', 'small', 'fuss', 'clean', 'well', 'keep', 'surprise', 'open', 'door', 'morning', 'good', 'service', 'wonderful', 'location', 'main', 'area', 'mean', 'nice', 'quiet', 'still', 'walkable', 'sight', 'would', 'stay']


TypeError: doc2bow expects an array of unicode tokens on input, not a single string